<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Harmonic_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kütüphaneler
!pip install git+https://github.com/rongardF/tvdatafeed backtesting
!pip install tradingview-screener==2.5.0
!pip installmatplotlib openpyxl matplotlib  TvDatafeed tqdm

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen zaman dilimi seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice = input("Seçiminizi yapın (1-9): ")
    return intervals.get(choice, ('4 Saat', Interval.in_4_hour))[1]

# TradingView API bağlantısı
print("TradingView oturum açılıyor...")
tv = TvDatafeed()

# BIST Hisse Senetleri
print("BIST hisse senetleri alınıyor...")
def get_bist_symbols():
    from tradingview_screener import get_all_symbols
    symbols = get_all_symbols(market='turkey')
    return sorted([symbol.replace('BIST:', '') for symbol in symbols])

hisseler = get_bist_symbols()

# Harmonik Formasyonları Tespiti

def identify_harmonic_patterns(data):
    patterns = []
    if len(data) < 5:
        return patterns

    for i in range(len(data) - 4):
        try:
            X, A, B, C, D = data['Close'][i:i+5]
            XA = A - X
            AB = B - A
            BC = C - B
            CD = D - C

            # Gartley Formasyonu
            if (
                abs((B - A) / XA - 0.618) < 0.05 and
                abs((C - B) / AB - 0.786) < 0.05 and
                abs((D - C) / BC - 0.786) < 0.05
            ):
                patterns.append(('Gartley', 'Bullish' if D > B else 'Bearish', i + 4, D * 1.27, D * 1.618))

            # Bat Formasyonu
            elif (
                abs((B - A) / XA - 0.5) < 0.05 and
                abs((C - B) / AB - 0.886) < 0.05 and
                abs((D - C) / BC - 0.886) < 0.05
            ):
                patterns.append(('Bat', 'Bullish' if D > B else 'Bearish', i + 4, D * 1.27, D * 1.618))

            # Butterfly Formasyonu
            elif (
                abs((B - A) / XA - 0.786) < 0.05 and
                abs((C - B) / AB - 1.272) < 0.05 and
                abs((D - C) / BC - 1.618) < 0.05
            ):
                patterns.append(('Butterfly', 'Bullish' if D > B else 'Bearish', i + 4, D * 1.618, D * 2.0))

            # Crab Formasyonu
            elif (
                abs((B - A) / XA - 0.382) < 0.05 and
                abs((C - B) / AB - 0.618) < 0.05 and
                abs((D - C) / BC - 1.618) < 0.05
            ):
                patterns.append(('Crab', 'Bullish' if D > B else 'Bearish', i + 4, D * 2.618, D * 3.618))

            # Cypher Formasyonu
            elif (
                abs((B - A) / XA - 0.382) < 0.05 and
                abs((C - B) / AB - 1.130) < 0.05 and
                abs((D - C) / BC - 0.786) < 0.05
            ):
                patterns.append(('Cypher', 'Bullish' if D > B else 'Bearish', i + 4, D * 1.272, D * 1.618))

            # AB=CD Formasyonu
            elif (
                abs((D - C) / (B - A) - 1.0) < 0.05
            ):
                patterns.append(('AB=CD', 'Bullish' if D > B else 'Bearish', i + 4, D * 1.272, D * 1.618))

            # Deep Crab Formasyonu
            elif (
                abs((B - A) / XA - 0.886) < 0.05 and
                abs((C - B) / AB - 0.382) < 0.05 and
                abs((D - C) / BC - 2.24) < 0.05
            ):
                patterns.append(('Deep Crab', 'Bullish' if D > B else 'Bearish', i + 4, D * 2.618, D * 3.618))

            # Double Top/Bottom
            elif (
                abs(A - C) / A < 0.02 and abs(B - D) / B < 0.02
            ):
                pattern_type = 'Double Top' if D < B else 'Double Bottom'
                patterns.append((pattern_type, 'Bearish' if D < B else 'Bullish', i + 4, D * 0.618, D * 0.382))

            # Head and Shoulders
            elif (
                abs(A - C) / A < 0.02 and B > A and D < C
            ):
                pattern_type = 'Head and Shoulders'
                patterns.append((pattern_type, 'Bearish', i + 4, D * 0.618, D * 0.382))

            # Symmetrical Triangle
            elif (
                abs(B - D) / B < 0.05 and abs(A - C) / A < 0.05
            ):
                patterns.append(('Symmetrical Triangle', 'Neutral', i + 4, None, None))

            # Ascending/Descending Triangle
            elif (
                abs(A - C) / A < 0.05 and B > D
            ):
                pattern_type = 'Ascending Triangle' if D > B else 'Descending Triangle'
                patterns.append((pattern_type, 'Bullish' if D > B else 'Bearish', i + 4, D * 1.272, D * 1.618))

            # Black Swan / White Swan
            elif (
                abs((B - A) / XA - 0.5) < 0.05 and
                abs((C - B) / AB - 1.618) < 0.05 and
                abs((D - C) / BC - 0.886) < 0.05
            ):
                pattern_type = 'Black Swan' if D < B else 'White Swan'
                patterns.append((pattern_type, 'Bullish' if D > B else 'Bearish', i + 4, D * 1.618, D * 2.0))
        except ZeroDivisionError:
            continue

    return patterns

# Grafik Çizim Fonksiyonu
def draw_harmonic_pattern(data, index, pattern_name, targets):
    plt.figure(figsize=(10, 6))
    prices = data['Close'][index-4:index+1]
    points = ['X', 'A', 'B', 'C', 'D']
    plt.plot(data.index[index-4:index+1], prices, marker='o', label=f'{pattern_name} Pattern')
    for i, point in enumerate(points):
        plt.text(data.index[index-4+i], prices[i], point, fontsize=12, color='red')

    # Fibonacci hedef seviyelerini çiz
    for target in targets:
        if target:
            plt.axhline(y=target, color='blue', linestyle='--', label=f'Target: {target:.2f}')

    plt.title(f"Harmonic Pattern Detected: {pattern_name}")
    plt.legend()
    plt.grid()
    plt.show()

# Ana Tarama Fonksiyonu

print("Zaman dilimi seçiliyor...")
interval = get_interval_choice()

results = []

print("BIST hisselerinde harmonic tarama başlıyor...")
for hisse in tqdm(hisseler, desc="Hisse Taraması", unit="hisse"):
    try:
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=interval, n_bars=500)
        if data is None or data.empty:
            print(f"Hata: {hisse} - Veri alınamadı.")
            continue

        data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        data = data.reset_index()

        patterns = identify_harmonic_patterns(data)
        for pattern_name, direction, index, target1, target2 in patterns:
            if target1 and target2:
                current_price = data.iloc[-1]['Close']
                if current_price < target2:
                    results.append((hisse, pattern_name, direction, data.iloc[index]['datetime'], target1, target2))
    except Exception as e:
        print(f"Hata: {hisse} - {e}")

# Sonuçların Listelenmesi
results_df = pd.DataFrame(results, columns=['Hisse', 'Formasyon', 'Tür', 'Tarih', 'Hedef 1', 'Hedef 2'])
results_df.to_excel('harmonic_patterns.xlsx', index=False)

print("\nHarmonic formasyonlar 'harmonic_patterns.xlsx' dosyasına kaydedildi.")

# Kullanıcı Seçimi ile Görsel Çıktı
while True:
    secim = input("Hisse seçin (çıkmak için 'q' yazın): ").strip()
    if secim.lower() == 'q':
        break

    secim_data = results_df[results_df['Hisse'] == secim]
    if secim_data.empty:
        print("Seçtiğiniz hisse için formasyon bulunamadı.")
        continue

    print(secim_data)

    for _, row in secim_data.iterrows():
        try:
            hisse_data = tv.get_hist(symbol=row['Hisse'], exchange='BIST', interval=interval, n_bars=500)
            hisse_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            draw_harmonic_pattern(hisse_data.reset_index(), row.name, row['Formasyon'], [row['Hedef 1'], row['Hedef 2']])
        except Exception as e:
            print(f"Görsel çizim hatası: {e}")

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-8kytfc1p
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-8kytfc1p
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
ERROR: unknown command "installmatplotlib"


TradingView oturum açılıyor...
Lütfen zaman dilimi seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Seçiminizi yapın (1-9): 7
BIST hisse senetleri alınıyor...
Formasyon taraması başlıyor...


Hisse Taraması: 100%|██████████| 584/584 [07:45<00:00,  1.25hisse/s]

Formasyonlar 'formasyonlar.xlsx' dosyasına kaydedildi.
